In [1]:
# Add parent directory
from sys import path as sysPath
sysPath.append('../')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic

sysPath.append('../Machine_Score/')
from Machine_Score import main_machine_score as ms

print("Hi!")

gm.test()
im.test()
ic.test()
ms.test()

Hi!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images


In [2]:
tDir = '/home/mbo2d/galStuff/spam_data_pl3/587722984435351614/' 
tInfo = im.target_info_class( targetDir = tDir, printAll=False)

rId = '00001'
rInfo = tInfo.getRunInfo(rID=rId, printAll = False)
rInfo.link_tInfo = tInfo
print(rInfo.status)

True


In [3]:
pLoc = gm.validPath('../param/param_base0_scores.json')
#print(pLoc)
newParams = im.group_score_parameter_class(pLoc = pLoc)
#gm.pprint(newParams.group)

In [4]:
runArgs = gm.inArgClass()
runArgs.setArg('newScore',True)
runArgs.printArg()
runArgs.setArg('scoreParams', newParams.group)

GM: Printing Input arguments
	- printBase - <class 'bool'> :  True
	- printAll - <class 'bool'> :  False
	- nProc - <class 'int'> :  1
	- simple - <class 'bool'> :  False
	- runDir - <class 'NoneType'> :  None
	- sdssDir - <class 'NoneType'> :  None
	- targetDir - <class 'NoneType'> :  None
	- dataDir - <class 'NoneType'> :  None
	- newScore - <class 'bool'> :  True


In [8]:

def pipelineRun( arg = None, rInfo = None ):

    # Initialize variables
    if arg == None:
        print("SIMR: WARNING: No arg. Exiting")

    rDir = arg.runDir
    params = arg.scoreParams
    printAll = arg.printAll
    printBase = arg.printBase

    if printBase:
        print("SIMR.pipelineRun: Inputs")
        print("\t - rDir:", rDir)
        print("\t - rInfo:", type(rInfo) )

    # Initialize info file
    if rInfo == None:
        rInfo = im.run_info_class( runDir=rDir, \
                printBase = printBase, printAll=printAll,\
                newInfo = newInfo, tInfoPtr=tInfoPtr )

    if printBase:
        print('SIMR.pipelineRun: ')
        print('\t - rInfo: ', (rInfo) )

    if rInfo.status == False:
        print("SIMR.pipelineRun: WARNING: runInfo bad")
        return

    if printBase:
        print("SIMR: run: scores before")
        rInfo.printScores()
    
    # Check if new files should be created/altered
    newSim = arg.get('newSim')
    newImg = arg.get('newImg')
    newScore = arg.get('newScore')
    newAll = arg.get('newAll')
    
    if newSim or newAll:
        if printBase: print("SIMR: run: newSim not functioning at this time")
        
    if newImg or newAll:
        if printBase: print("SIMR: run: newImg not functioning at this time")

    if newScore or newAll:

        MS_Run( printBase = printBase, printAll = printAll, \
                rInfo = rInfo, params = arg.get('scoreParams'), \
                arg = arg )

# end processing run

pipelineRun( arg = runArgs, rInfo=rInfo)


SIMR.pipelineRun: Inputs
	 - rDir: None
	 - rInfo: <class 'Support_Code.info_module.run_info_class'>
SIMR.pipelineRun: 
	 - rInfo:  <Support_Code.info_module.run_info_class object at 0x7f7c74f703d0>
SIMR: run: scores before
IM: run_info_class.printScores()
	 - run_id: r00001
	 - zoo_merger: 0.941176
	 - test_v0: 0.737633
	 - test_perturbation: 0.716723
MS: main_run:
MODEL NAME:  default
Model loc:  /home/mbo2d/galStuff/spam_data_pl3/587722984435351614/gen000/run_00001/model_images/default_model.png


In [9]:

# Create scores and comparison scores for run. 
def MS_Run( \
        printBase=True, printAll=False, \
        runDir = None, rInfo = None, \
        params = None, arg = None, \
        ):

    if printBase: 
        print("MS: main_run:")

    if rInfo == None:
        rInfo = im.run_info_class( runDir=runDir, printAll=printAll, )

    # Check if successfully read info data
    if rInfo.status == False:
        if printBase:
            print("MS: Error: pipelineRun: Base status")
            print("\t- rInfo: ", rInfo.status)
        return None

    # Check if params were given
    if params == None:
        if printBase:
            print("MS: Error: pipelineRun: Bad score parameters")
            print("\t- params: ", params)
        return None

    # Check if params were given
    if arg == None:
        if printBase:
            print("MS: Error: pipelineRun: Bad argument input")
            print("\t- arg type: ", type(arg) )
        return None

    # Assume group of parameters and loop through.
    
    for pKey in params:
        
        # Check if score exists
        score = rInfo.getScore( pKey )
        
        # If score exists, move to next score parameter
        if score != None:
            continue
        
        # Grab parameter and score type
        param = params[pKey]
        scoreType = param['scoreType']
        
        # Call function with correct score type
        if scoreType == 'target':
            target_image_compare(rInfo, param, arg) 
        
        # focus on one score for now
        break
    
# end processing run dir

# TODO
def target_image_compare(rInfo, param, args):
    
    printBase = args.printBase
    
    # GET Target Image
    tName = param['targetName'] 
    tLoc = None
    tImg = None
    tLink = rInfo.get('link_tInfo')
    
    # Create dict for storing target and model images if needed
    if rInfo.get('targetImg') == None:
        rInfo.targetImg = {}
        
    if rInfo.get('modelImg') == None:
        rInfo.modelImg = {}
        
    # Check if in rInfo has target Image already
    if type( rInfo.targetImg.get(tName) ) != type(None):
        tImg = rInfo.targetImg.get(tName)
    
    # Check other source
    if type(tLoc) == type(None) and type(tLink) == type(None):
        if printBase:
                print("MS: Error: target_image_compare: no target image or link given")
        return None
    
    # If given tLink
    if type(tLoc) == type(None):
        tLoc = tLink.findTargetImage(tName)
        tImg = gm.readImg(tLoc)
    
        # Finally, should have tImg.  Leave if not
        if type(tImg) != None:
            rInfo.targetImg[tName] = tImg
        else:
            if printBase:
                print("MS: Error: target_image_compare: failed to load target image")
            return None
    
    # GET Model Image
    
    mName = param['imgArg']['name']
    mImg = None
    mloc = None
    print("MODEL NAME: ",mName)
        
    # Check if in rInfo has model Image already
    if type( rInfo.modelImg.get(mName) ) != type(None):
        mImg = rInfo.modelImg.get(mName)
    
    # Else, have rInfo retrieve img location
    else:
        mLoc = rInfo.findImgLoc(mName)
        print("Model loc: ",mLoc)
        if gm.validPath(mLoc):
            mImg = gm.readImg(mLoc)
        
    if type(mImg) == type(None): 
            if printBase: 
                print("MS: Error: target_image_compare: failed to load model image")
                gm.tabprint("model: "%mName)
            return None
    
    # Create score and add to rInfo
    
    score = ms.createScore( tImg, mImg, cmpMethod= param['cmpArg']['cmpMethod'] )
    newScore = rInfo.addScore( name = param['name'], score=score )

    rInfo.saveInfoFile()
    return score

    
    return None

# TODO
def perturbation_image_compare(rInfo, param, args):
    print(param)           
        
    tLoc = rInfo.findImgFile( pClass.get('imgArg')['name'], initImg=True )

    return None

pipelineRun( arg = runArgs, rInfo=rInfo)


SIMR.pipelineRun: Inputs
	 - rDir: None
	 - rInfo: <class 'Support_Code.info_module.run_info_class'>
SIMR.pipelineRun: 
	 - rInfo:  <Support_Code.info_module.run_info_class object at 0x7f7c74f703d0>
SIMR: run: scores before
IM: run_info_class.printScores()
	 - run_id: r00001
	 - zoo_merger: 0.941176
	 - test_v0: 0.737633
	 - test_perturbation: 0.716723
MS: main_run:
MODEL NAME:  default
Model loc:  /home/mbo2d/galStuff/spam_data_pl3/587722984435351614/gen000/run_00001/model_images/default_model.png


NameError: name 'self' is not defined